# General structure analysis

https://rnasolo.cs.put.poznan.pl/api/query/structure?pdbid=7KUB&format=PDB&chains=A

In [143]:
import requests as r
from Bio.PDB import PDBParser
from Bio import SeqIO
from io import StringIO
from pprint import pprint
import numpy as np
import re

In [151]:
pdb_id = "7T1N"
structure_chains = "A"

In [152]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## RNA structure request

In [153]:
def get_raw_RNA_data(pdb_id, structure_chains, format):
    url = f'https://rnasolo.cs.put.poznan.pl/api/query/structure?pdbid={pdb_id}&format={format}&chains={structure_chains}'
    response = r.get(url, verify=False)
    return response.text

In [154]:
raw_fasta_format = get_raw_RNA_data(pdb_id=pdb_id, structure_chains=structure_chains, format="FASTA")
pprint(raw_fasta_format)

'{"results": "Invalid request - No result"}'


In [148]:
raw_PDB_format = get_raw_RNA_data(pdb_id=pdb_id, structure_chains=structure_chains, format="PDB")
pprint(raw_PDB_format)

('REMARK '
 '300                                                                     \n'
 'REMARK 300 BIOMOLECULE: 1\n'
 'REMARK 300 SEE REMARK 350 FOR THE AUTHOR PROVIDED AND/OR '
 'PROGRAM               \n'
 'REMARK 300 GENERATED ASSEMBLY INFORMATION FOR THE STRUCTURE '
 'IN                 \n'
 'REMARK 300 THIS ENTRY. THE REMARK MAY ALSO PROVIDE INFORMATION '
 'ON              \n'
 'REMARK 300 BURIED SURFACE '
 'AREA.                                                \n'
 'REMARK '
 '350                                                                     \n'
 'REMARK 350 COORDINATES FOR A COMPLETE MULTIMER REPRESENTING THE '
 'KNOWN          \n'
 'REMARK 350 BIOLOGICALLY SIGNIFICANT OLIGOMERIZATION STATE OF '
 'THE               \n'
 'REMARK 350 MOLECULE CAN BE GENERATED BY APPLYING BIOMT '
 'TRANSFORMATIONS         \n'
 'REMARK 350 GIVEN BELOW.  BOTH NON-CRYSTALLOGRAPHIC '
 'AND                         \n'
 'REMARK 350 CRYSTALLOGRAPHIC OPERATIONS ARE '
 'GIVEN.                        

## parsing RNA secondary structure

In [149]:
def return_secondary_structures_from_FASTA_format(raw_format):
    handle = StringIO(raw_format)
    records = list(SeqIO.parse(handle, "fasta"))
    secondary_structures = {}
    for record in records:
        structure_chain_idx = record.id.split("_")[-1]
        secondary_structures[structure_chain_idx] = str(record.seq)
    return secondary_structures

In [150]:
return_secondary_structures_from_FASTA_format(raw_format=raw_fasta_format)

{'A': 'GUGAACCGGAUUGCCGUCAGGAAAUUAGGUGCGCUUAGCAGUAUUCCCCACGCUGUGGG',
 'M': 'GCTAAGCGCACCTAATTTCCTGACGGCAATCCT'}

## parsing RNA tertiary structure

In [138]:
def return_tertiary_structure_from_PDB_format(raw_format):
    """
    Parse an RNA PDB file (as a raw text string) and return a list of rows.
    Each row contains: [nucleotide, x, y, z] using only the C5' atom.
    The function applies any BIOMT transformations found in the file.
    
    Parameters:
      raw_format (str): The full text of a PDB file.
    
    Returns:
      list: A list of lists, each inner list is [nucleotide, x, y, z].
    """
    # ---------------------------
    # Step 1: Parse the PDB structure from the raw text.
    # ---------------------------
    pdb_handle = StringIO(raw_format)
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_handle)

    # ---------------------------
    # Step 2: Parse BIOMT records from the raw text.
    # ---------------------------
    biomt_lines = [
        line for line in raw_format.splitlines() 
        if line.startswith("REMARK 350") and "BIOMT" in line
    ]

    # Group BIOMT lines by transformation number.
    transformations = {}
    for line in biomt_lines:
        parts = line.split()
        # Expected parts: ['REMARK', '350', 'BIOMT1', '1', '1.000000', '0.000000', '0.000000', '0.000000']
        biomt_label = parts[2]  # e.g., "BIOMT1"
        # Use regex to extract the row number (e.g., 1, 2, or 3)
        match = re.search(r"BIOMT(\d+)", biomt_label)
        if not match:
            continue
        row_index = int(match.group(1))
        try:
            trans_num = int(parts[3])
        except ValueError:
            continue
        m_row = [float(parts[4]), float(parts[5]), float(parts[6])]
        t_val = float(parts[7])
        if trans_num not in transformations:
            transformations[trans_num] = {}
        transformations[trans_num][row_index] = (m_row, t_val)

    # Build complete 4x4 transformation matrices.
    transformation_matrices = []
    if len(transformations) == 0:
        transformation_matrices.append(np.eye(4))
    else:
        for trans_num in sorted(transformations.keys()):
            if len(transformations[trans_num]) != 3:
                continue
            T = np.eye(4)
            for i in [1, 2, 3]:
                m_row, t_val = transformations[trans_num][i]
                T[i - 1, 0:3] = m_row
                T[i - 1, 3] = t_val
            transformation_matrices.append(T)

    # ---------------------------
    # Step 3: Loop over residues and extract the coordinate for the C5' atom.
    # ---------------------------
    data = []  # Each row: [nucleotide, x, y, z]
    for model in structure:
        for chain in model:
            for residue in chain:
                # Get the residue name (e.g., "C", "U", etc.)
                nucleotide = residue.get_resname().strip()
                # Check if the residue contains the C5' atom
                if "C5'" not in residue:
                    continue
                atom = residue["C5'"]
                coord = atom.get_coord()  # (x, y, z) as a numpy array
                # Convert to homogeneous coordinates [x, y, z, 1]
                v = np.append(coord, 1.0)
                # Apply each transformation (usually just one, identity if none provided)
                for T in transformation_matrices:
                    v_trans = T @ v  # Matrix multiplication
                    x, y, z, _ = v_trans
                    x, y, z = float(x), float(y), float(z)
                    data.append([nucleotide, x, y, z])
    return data

In [139]:
def return_tertiary_structure_from_PDB_format(raw_format):
    """
    Parse an RNA PDB file (as a raw text string) and return a dictionary mapping chain IDs 
    to a NumPy array. Each array has rows of the form [nucleotide, x, y, z] for the C5' atom.
    The function applies any BIOMT transformations found in the file.

    Parameters:
      raw_format (str): The full text of a PDB file.
    
    Returns:
      dict: A dictionary where keys are chain IDs and values are NumPy arrays of shape (n, 4)
            with rows [nucleotide, x, y, z].
    """
    import numpy as np
    import re
    from Bio.PDB import PDBParser
    from io import StringIO

    # ---------------------------
    # Step 1: Parse the PDB structure from the raw text.
    # ---------------------------
    pdb_handle = StringIO(raw_format)
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_handle)

    # ---------------------------
    # Step 2: Parse BIOMT records from the raw text.
    # ---------------------------
    biomt_lines = [
        line for line in raw_format.splitlines() 
        if line.startswith("REMARK 350") and "BIOMT" in line
    ]
    # Group BIOMT lines by transformation number.
    transformations = {}
    for line in biomt_lines:
        parts = line.split()
        # Expected parts: ['REMARK', '350', 'BIOMT1', '1', '1.000000', '0.000000', '0.000000', '0.000000']
        biomt_label = parts[2]  # e.g., "BIOMT1"
        # Use regex to extract the row number (1, 2, or 3)
        match = re.search(r"BIOMT(\d+)", biomt_label)
        if not match:
            continue
        row_index = int(match.group(1))
        try:
            trans_num = int(parts[3])
        except ValueError:
            continue
        m_row = [float(parts[4]), float(parts[5]), float(parts[6])]
        t_val = float(parts[7])
        if trans_num not in transformations:
            transformations[trans_num] = {}
        transformations[trans_num][row_index] = (m_row, t_val)

    # Build complete 4x4 transformation matrices.
    transformation_matrices = []
    if len(transformations) == 0:
        transformation_matrices.append(np.eye(4))
    else:
        for trans_num in sorted(transformations.keys()):
            if len(transformations[trans_num]) != 3:
                continue
            T = np.eye(4)
            for i in [1, 2, 3]:
                m_row, t_val = transformations[trans_num][i]
                T[i - 1, 0:3] = m_row
                T[i - 1, 3] = t_val
            transformation_matrices.append(T)

    # ---------------------------
    # Step 3: Loop over chains and residues, grouping results by chain.
    # ---------------------------
    data_by_chain = {}
    for model in structure:
        for chain in model:
            chain_id = chain.id
            chain_coords = []  # Will store rows for this chain
            for residue in chain:
                # Get the residue name (e.g., "C", "U", etc.)
                nucleotide = residue.get_resname().strip()
                # Check if the residue contains the C5' atom.
                if "C5'" not in residue:
                    continue
                atom = residue["C5'"]
                coord = atom.get_coord()  # numpy array: (x, y, z)
                # Convert to homogeneous coordinate: [x, y, z, 1]
                v = np.append(coord, 1.0)
                # Apply each transformation matrix.
                for T in transformation_matrices:
                    v_trans = T @ v  # Matrix multiplication
                    x, y, z, _ = v_trans
                    # Append the row; nucleotide remains as a string.
                    chain_coords.append([nucleotide, float(x), float(y), float(z)])
            # Convert the list to a NumPy array (dtype=object allows mixing strings and floats)
            if chain_coords:
                data_by_chain[chain_id] = np.array(chain_coords, dtype=object)
            else:
                data_by_chain[chain_id] = np.empty((0, 4), dtype=object)
    return data_by_chain

In [140]:
return_tertiary_structure_from_PDB_format(raw_PDB_format)

{'k': array([['A', 37.880001068115234, 5.77400016784668, -11.904999732971191],
        ['A', -23.94042902941704, 29.91802784109116, -11.904999732971191],
        ['A', -13.939572038698195, -35.69202800893784,
         -11.904999732971191],
        ['A', 35.47700119018555, 5.90500020980835, -17.701000213623047],
        ['A', -22.85237840179205, 27.771469850826264, -17.701000213623047],
        ['A', -12.624622788393498, -33.676470060634614,
         -17.701000213623047],
        ['A', 35.93299865722656, 7.066999912261963, -23.749000549316406],
        ['A', -24.08669792762995, 27.585375205993653, -23.749000549316406],
        ['A', -11.846300729596614, -34.65237511825562,
         -23.749000549316406],
        ['A', 39.03499984741211, 7.572999954223633, -29.22800064086914],
        ['A', -26.075907209062578, 30.01878576574326, -29.22800064086914],
        ['A', -12.959092638349532, -37.59178571996689, -29.22800064086914],
        ['A', 43.65399932861328, 5.3480000495910645, -32.8940010

# Collecting data id information for us

[('2LKR', 'A'), ('7KUB', 'A'), ('7KUC', 'A'), ('7KUD', 'A'), ('7ZEX', 'B'), ('7QA2', 'A'), ('7T1O', 'A'), ('7T1N', 'A'), ('8FCS', 'A'), ('7Q4L', 'B'), ('7UGA', 'A'), ('7RWR', 'A'), ('9CDB', 'J'), ('7SHX', 'A'), ('7ZEW', 'B'), ('7QDD', 'A'), ('7Q33', 'B'), ('4CIO', 'B'), ('7UMD', 'A'), ('7PS8', 'A'), ('7UME', 'A'), ('7UMC', 'A'), ('7V06', 'A'), ('8SCH', 'A'), ('8K15', 'C'), ('8I45', 'A'), ('7VRL', 'A'), ('9CPG', 'A,B'), ('7QDE', 'A'), ('7ZAP', 'B'), ('9CPI', 'A,B'), ('8SCF', 'A'), ('8I43', 'A'), ('8THV', 'A'), ('1HG9', 'B'), ('7Q6L', 'A'), ('8K8B', 'A'), ('8BWT', 'A'), ('9CPJ', 'A,B'), ('9BCI', 'A'), ('8CQ1', 'A'), ('9FO8', 'A'), ('8RDU', '1'), ('8JHP', 'A'), ('8Z97', 'F,G'), ('8WUV', 'B'), ('6N6H', 'D'), ('7Q48', 'A'), ('7FGI', 'M'), ('8JY0', 'B'), ('9CPD', 'A,B'), ('8KAH', 'I'), ('8WUT', 'B'), ('8UO6', 'B'), ('7NEJ', 'B'), ('2XEB', 'A,B'), ('4A4T', 'A'), ('8JTJ', 'B'), ('8UO6', 'A'), ('8JTR', 'B'), ('8GLP', 'L5,L8'), ('8KAL', 'A'), ('8CLR', 'A'), ('4V3P', 'S3'), ('8Z98', 'D'), ('3WFS'